In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
from stl import mesh
import matplotlib.pyplot as plt

In [3]:
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(fr'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [4]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]


In [5]:
a = 3.4
lambdas = np.array([8,3.5])/0.8
# radial distance away from the origin at which to project fields
r_proj =  1e6

# theta and phi angles at which to observe fields - part of the half-space to the right
theta_proj = np.linspace(0, np.pi/2, 280)
phi_proj = np.linspace(0,2*np.pi, 280)

In [ ]:
# folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5\20250116"
folder_path = rf"H:\phd stuff\tidy3d\structures\LSU H5\20241208"
project_name = "20250328 Far Field Transmission LSU Angle Test 2073_ff_4x_size"
runtime_ps = 25e-12
min_steps_per_lambda = 20
ref = True
cuts = [1]
h5_bg = None
for direction in ["z"]: 
    for f,filename in enumerate(os.listdir(folder_path)):
     if filename not in ["ak4_1000_eps_9p0_phih5_0p2177_phiL_0p2073.h5"]:
        continue
     for cut in cuts:
        if not (Path(filename).suffix==".h5" or Path(filename).suffix==".stl"):
            continue 
        if os.path.isfile(os.path.join(folder_path, filename)):
            file=os.path.join(folder_path, filename)
            structure_1 = AM.loadAndRunStructure(key = tidy3dAPI, file_path=file
                                            ,direction=direction, lambda_range=lambdas,
                                            box_size=14.3,runtime_ps=runtime_ps,min_steps_per_lambda=min_steps_per_lambda,
                                           scaling=1,shuoff_condtion=1e-20, verbose=True, multiplicate_size=True,multiplication_factor=4,
                                           monitors=["flux", "far_field"], flux_monitor_position=12.5,cell_size_manual=35,
                                           freqs=450, far_field_settings={"r":r_proj, "theta":theta_proj,"phi":phi_proj, "window_size":0.25},
                                           cut_condition=cut, source="gaussian", gaussian_params={"waist_radius":5.2,"waist_distance":-25,"theta":15*np.pi/180,"phi":0, "position_x":-21, "size":12}, 
                                           absorbers=130, boundaries= "absorbers",
                                            use_permittivity=False,sim_name=rf"{Path(filename).stem}_size_{cut}" + (rf"_bg_{h5_bg}" if h5_bg else ""),h5_bg=h5_bg,
                                           )
        
        file_desc = rf"H:\phd stuff\tidy3d\data\{project_name}_perm_{structure_1.permittivity_value}\z_incidence\{structure_1.sim_name}.txt"
        if os.path.exists(file_desc):
            print("Exist!")
        else:
           print("Creating...")
           structure_1.run_sim(run_free=False,load=False,add_ref=ref,folder_description=rf"{project_name}_perm_{structure_1.permittivity_value}",monitor=True)
           if ref:
              ref = False

        del structure_1


        # structure_1.estimate_cost()
        # structure_1.sim.plot_3d()
        # raise Exception


In [7]:
import numpy as np
np.sin((lambdas[0])/(np.pi*5))

0.5944807685248221

In [8]:
(7.5*np.pi/180)

0.1308996938995747

In [9]:
lambdas

array([10.   ,  4.375])

In [16]:
lambdas[1]/(np.pi*(0.131))

10.630578259954842